In [4]:
# 1. Import .py files:
from core.imports import *
from core.data_preprocessing import load_mnist, preprocess_data
from core.custom_confusion_matrix import custom_confusion_matrix
from core.multi_class_perceptron import MultiClassPerceptron
from core.evaluation_functions import evaluate_model, plot_confusion_matrix

# 2. Load and preprocess data:
X, y = load_mnist()
X = preprocess_data(X)

# 3. Split into train/test:
X_train, X_test = X[:60000], X[60000:]
y_train, y_test = y[:60000], y[60000:]

# 4. Train the model:
mcp = MultiClassPerceptron()
mcp.fit(X_train, y_train)

# 5. Evaluate:
cm, acc = evaluate_model(mcp, X_test, y_test)
plot_confusion_matrix(cm)


ModuleNotFoundError: No module named 'custom_confusion_matrix'

In [ ]:
from core.imports import *
%matplotlib inline

# 1. Ensure the outputs directory exists
os.makedirs("outputs", exist_ok=True)

# 2. Generate unique filenames for this run (using a timestamp)
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
log_filename = f"outputs/run_{timestamp}_log.txt"
img_filename = f"outputs/run_{timestamp}_confusion_mat.png"

# 3. Redirect stdout to a new log file
with open(log_filename, "w") as f:
    original_stdout = sys.stdout
    sys.stdout = f  # All print statements will go to run_{timestamp}_log.txt

    print("=== Starting a new run ===")
    print(f"Log file: {log_filename}")
    print(f"Image file: {img_filename}")
    
    # -- Import or reference your code modules if needed --
    from core.data_preprocessing import load_mnist, preprocess_data
    from core.custom_confusion_matrix import custom_confusion_matrix
    from core.multi_class_perceptron import MultiClassPerceptron
    from core.evaluation_functions import evaluate_model, plot_confusion_matrix

    # -- Load and preprocess data --
    print("=== Loading and Preprocessing Data ===")
    X, y = load_mnist()
    X = preprocess_data(X)

    # -- Split into train/test --
    print("=== Splitting Data ===")
    X_train, X_test = X[:60000], X[60000:]
    y_train, y_test = y[:60000], y[60000:]

    # -- Train the model --
    print("=== Training MultiClassPerceptron ===")
    mcp = MultiClassPerceptron()
    mcp.fit(X_train, y_train)

    # -- Evaluate on test set --
    print("=== Evaluating on Test Set ===")
    cm, acc = evaluate_model(mcp, X_test, y_test)

    # Restore stdout
    sys.stdout = original_stdout

# 4. Plot and save the confusion matrix image
plot_confusion_matrix(cm)         # This displays in the notebook
plt.savefig(img_filename)         # Save to e.g. outputs/run_20230305_123456_confusion_mat.png
plt.show()                        # Show inline in the notebook

print(f"Run complete. Logs saved to {log_filename} and figure saved to {img_filename}.")
